# Theory

- Every animal is made up of different types of cells (e.g., muscle cells, nerve cells, blood cells).
- Almost all cells contain a nucleus, which acts as the control center of the cell.
- Inside the nucleus, cells contain genomes. The human genome consists of 23 pairs of chromosomes.
- Chromosomes are composed of genes, which are the basic functional units of heredity:
- Genes contain instructions for making proteins and other essential molecules. They act as "recipes" that guide various cellular functions. While the complete set of genes is present in almost all cells, only certain genes are expressed (active) in each cell type. This selective gene expression is similar to having a complete blueprint of a house in every room, but only using specific parts as needed for each room.
- Genes are made up of DNA (Deoxyribonucleic Acid). DNA is composed of units called nucleotides.
- There are four types of nucleotides in DNA:
  - Adenine (A)
  - Guanine (G)
  - Cytosine (C)
  - Thymine (T)
- The sequence of these nucleotides determines the genetic information carried by the DNA.
- Genes are made up of DNA sequences. However, not all parts of a gene are used directly to make proteins. Within a gene, there are coding regions called exons, and non-coding regions called introns. Exons contain the actual instructions (code) for making proteins, while introns do not.

# Prelimnaries

In [1]:
import os

# Data preparation

## Data Files Description

1. chrX.fa:
   - Contains the reference sequence for Chromosome X in FASTA format
   - First line is the header, followed by the sequence (newlines can be ignored)
   - Approximately 150 million base pairs long

2. chrX_last_col.txt:
   - Contains the last column of the Burrows-Wheeler Transform (BWT) for Chromosome X
   - This sequence has one more character than the reference sequence
   - Includes a special '$' character appended to its end

3. chrX_map.txt:
   - Maps indexes in BWT to indexes in the reference sequence for Chromosome X
   - Each line number i (0-based) contains the starting index in the reference of the i-th sorted circular shift
   - Example: If the first line contains 3435, it means the string starting at index 3435 (0-based) in the reference is the first in sort order of BWT rotated strings

4. reads:
   - Contains approximately 3 million reads from the genome
   - One read per line
   - Reads are approximately 100 characters long, but may vary slightly
   - Each read could come from the reference sequence or its reverse complement
   - Consider both original and reverse complement versions of each read
   - These 3 million reads are a subset of the 150 million reads actually generated

5. Red and Green Gene Locations in Chromosome X:
   - Both genes should begin with the sequence: ATGGCCCAGCAGTGGAGCCTC
   - You can use 'grep' to find the starting positions (0-based)

   Red Gene Exons:
   - 149249757 - 149249868
   - 149256127 - 149256423
   - 149258412 - 149258580
   - 149260048 - 149260213
   - 149261768 - 149262007
   - 149264290 - 149264400

   Green Gene Exons:
   - 149288166 - 149288277
   - 149293258 - 149293554
   - 149295542 - 149295710
   - 149297178 - 149297343
   - 149298898 - 149299137
   - 149301420 - 149301530

## Load refrence sequence

In [2]:
def load_reference_sequence(file_path):
    """
    Load the reference sequence from a FASTA file using a more efficient approach.

    Args:
    file_path (str): The absolute path to the FASTA file.

    Returns:
    str: The reference sequence as a single string.
    """
    sequence_fragments = []  # Use a list to store sequence fragments for efficient concatenation
    with open(file_path, 'r') as file:  # Open the file in read mode
        next(file)  # Skip the header line of the FASTA file
        for line in file:  # Iterate through each line in the file
            sequence_fragments.append(line.strip())  # Append the stripped line to the list
    return ''.join(sequence_fragments)  # Join the list into a single string and return the sequence

# Define the absolute path to the reference sequence file
reference_file_path = r"D:\Users\VICTOR\Desktop\Data analytics\Assignment 5\chrX_bwt\chrX.fa"

# Load the reference sequence
reference_sequence = load_reference_sequence(reference_file_path)  # Call the function to load the sequence

In [3]:
# Print some basic information about the loaded sequence
print(f"Reference sequence loaded.")
print(f"Length of reference sequence: {len(reference_sequence)} base pairs")
print(f"First 50 base pairs: {reference_sequence[:50]}")
print(f"Last 50 base pairs: {reference_sequence[-50:]}")

Reference sequence loaded.
Length of reference sequence: 151100560 base pairs
First 50 base pairs: CTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTCTGAAAGTGGAC
Last 50 base pairs: GTGGGTGTGGGTGTGGGTGTGGGTGTGTGGGTGTGGTGTGTGGGTGTGGT


## Load reads

In [4]:
def load_reads(file_path):
    """
    Load reads from a file.

    Args:
    file_path (str): The path to the file containing the reads.

    Returns:
    list: A list of strings, where each string is a read from the file.
    """
    reads = []  # Initialize an empty list to store the reads
    with open(file_path, 'r') as file:  # Open the file in read mode
        for line in file:  # Iterate through each line in the file
            read = line.strip()  # Remove any leading/trailing whitespace
            if read:  # Check if the read is not empty
                reads.append(read)  # Add the read to the list
    return reads  # Return the list of reads

# Define the path to the reads file
reads_file_path = r"D:\Users\VICTOR\Desktop\Data analytics\Assignment 5\chrX_bwt\reads"

# Load the reads from the file
reads = load_reads(reads_file_path)  # Call the load_reads function to get the list of reads

# Print some information about the loaded reads
print(f"Total number of reads loaded: {len(reads)}")  # Print the total number of reads
print(f"First 5 reads: {reads[:5]}")  # Print the first 5 reads as a sample
print(f"Length of first read: {len(reads[0])}")  # Print the length of the first read


Total number of reads loaded: 3066720
First 5 reads: ['CTTTTGTTTTTTTTTTTTTTTTTTTATAAAATATTTGGGTTTTATAAAAAAAAACACAAAAAAAAAAAAAAAAAAAAAAAACTGAACCTATCGGATGACGT', 'GTCCCAGAGTTTTTCTTCATCACTCAGTATTTGGCAACAGGGAAGTTTTTCTAAGCAAGTCAACCCATCAATGTTAGACTGGATAAAGAAAATGCGGCACA', 'TACCCCTGCATGCATTCTTTCCACCCGACCCCCTCAAATACCCACTTATAGTACACAACCCACATGTCATCTTTTTCCAACTTACGTGTCTCACTCTCTAA', 'CACACGTTTGGAAGATCCTTAGAGTCTATTGAAACTGCAAAGATCCCGGAGCTGGTCTCCGATGAAAATGCCATTTCTTCGTTGCCAACGATTTTCTTTAT', 'TTTCTTTTTTTTTTTTTTTTGTTTTTTTAAAGCCCTTGTTTGGGTTGAAAATAAAGGTTTAAATAGCTCATAAATTTATGGGTCTCTTAATATCTAAAACC']
Length of first read: 101


## Load last column

In [5]:
def load_bwt_last_column(file_path):
    """
    Load the BWT last column from a text file.

    Args:
    file_path (str): The absolute path to the BWT last column file.

    Returns:
    str: The BWT last column as a single string.
    """
    with open(file_path, 'r') as file:  # Open the file in read mode
        return file.read().strip()  # Read the entire file, strip whitespace, and return

# Define the absolute path to the BWT last column file
bwt_last_column_path = r"D:\Users\VICTOR\Desktop\Data analytics\Assignment 5\chrX_bwt\chrX_last_col.txt"

# Load the BWT last column
bwt_last_column = load_bwt_last_column(bwt_last_column_path)  # Call the function to load the BWT last column

# Print some basic information about the loaded BWT last column
print(f"BWT last column loaded.")
print(f"Length of BWT last column: {len(bwt_last_column)} characters")
print(f"First 50 characters: {bwt_last_column[:50]}")
print(f"Last 50 characters: {bwt_last_column[-50:]}")

BWT last column loaded.
Length of BWT last column: 152611566 characters
First 50 characters: CAAAAAAAAAAAACAATAAAAAATAACAAACAACCACAAAAAAAAGATAC
Last 50 characters: TTTTTTTTTTTTTTTTTCTTTTTTTTTTCTTGTTTTATTTTGTGTTTCGT


## Load bwt map

In [6]:
def load_bwt_map(file_path):
    """
    Load the BWT to reference sequence mapping from a file.

    Args:
    file_path (str): The absolute path to the chrX_map.txt file.

    Returns:
    list: A list where index i contains the reference sequence index for the i-th sorted rotation in BWT.
    """
    bwt_map = []  # Initialize an empty list to store the mapping
    with open(file_path, 'r') as file:  # Open the file in read mode
        for line in file:  # Iterate through each line in the file
            bwt_map.append(int(line.strip()))  # Convert each line to an integer and append to the list
    return bwt_map  # Return the completed mapping list

# Define the absolute path to the chrX_map.txt file
map_file_path = r"D:\Users\VICTOR\Desktop\Data analytics\Assignment 5\chrX_bwt\chrX_map.txt"  # Absolute path to the chrX_map.txt file

# Load the BWT to reference sequence mapping
bwt_map = load_bwt_map(map_file_path)  # Call the load_bwt_map function with the specified file path

# Print some basic information about the loaded mapping
print(f"BWT to reference sequence mapping loaded.")  # Indicate that the mapping has been loaded
print(f"Length of mapping: {len(bwt_map)}")  # Print the total number of entries in the mapping
print(f"First 5 entries: {bwt_map[:5]}")  # Print the first 5 entries of the mapping
print(f"Last 5 entries: {bwt_map[-5:]}")  # Print the last 5 entries of the mapping

BWT to reference sequence mapping loaded.
Length of mapping: 151100561
First 5 entries: [74645898, 74645899, 74645900, 74645901, 74645902]
Last 5 entries: [39459922, 39459921, 39459920, 151100559, 151100560]


In [7]:
def bwt_to_reference_position(bwt_position, bwt_map):
    """
    Convert a position in the BWT to its corresponding position in the reference sequence.

    Args:
    bwt_position (int): The position in the BWT.
    bwt_map (list): The BWT to reference sequence mapping.

    Returns:
    int: The corresponding position in the reference sequence.
    """
    return bwt_map[bwt_position]  # Return the reference sequence position from the mapping

# Example usage
bwt_pos = 1000000  # Example BWT position
ref_pos = bwt_to_reference_position(bwt_pos, bwt_map)  # Convert BWT position to reference position
print(f"BWT position {bwt_pos} corresponds to reference position {ref_pos}")  # Print the result of the conversion

BWT position 1000000 corresponds to reference position 109615724


# Auxilary datastructures

## Rank datastructure

In [8]:
def create_rank_structure(bwt_last_column, checkpoint_interval=100000):
    """
    Create a memory-efficient rank data structure for fast occurrence counting in alignment.
    
    Args:
    bwt_last_column (str): The last column of the BWT
    checkpoint_interval (int): Interval for storing checkpoints
    
    Returns:
    dict: A dictionary with characters as keys and lists of cumulative counts as values
    """
    unique_chars = set(bwt_last_column)  # Get unique characters once to avoid repeated set() calls
    rank = {char: [] for char in unique_chars}  # Initialize rank structure with empty lists for each unique character
    counts = {char: 0 for char in unique_chars}  # Initialize a count dictionary for each character
    
    for i, char in enumerate(bwt_last_column, 1):  # Iterate through each character in the BWT last column, starting from 1
        counts[char] += 1  # Increment the count for the current character
        if i % 1000000 == 0:  # Check if we've processed 1,000,000 characters
            print(f"Processed {i} characters")  # Print progress every 1,000,000 characters
        if i % checkpoint_interval == 0:  # Check if we've reached a checkpoint
            for c, count in counts.items():  # Iterate through counts dictionary
                rank[c].append(count)  # Append the current count to the rank structure
    
    # Add final checkpoint if not already added
    if len(bwt_last_column) % checkpoint_interval != 0:
        for c, count in counts.items():
            rank[c].append(count)
    
    return rank

In [9]:
# Create the rank structure for efficient occurrence counting
rank_structure = create_rank_structure(bwt_last_column)  # Use the previously defined function to create the rank structure

Processed 1000000 characters
Processed 2000000 characters
Processed 3000000 characters
Processed 4000000 characters
Processed 5000000 characters
Processed 6000000 characters
Processed 7000000 characters
Processed 8000000 characters
Processed 9000000 characters
Processed 10000000 characters
Processed 11000000 characters
Processed 12000000 characters
Processed 13000000 characters
Processed 14000000 characters
Processed 15000000 characters
Processed 16000000 characters
Processed 17000000 characters
Processed 18000000 characters
Processed 19000000 characters
Processed 20000000 characters
Processed 21000000 characters
Processed 22000000 characters
Processed 23000000 characters
Processed 24000000 characters
Processed 25000000 characters
Processed 26000000 characters
Processed 27000000 characters
Processed 28000000 characters
Processed 29000000 characters
Processed 30000000 characters
Processed 31000000 characters
Processed 32000000 characters
Processed 33000000 characters
Processed 34000000 

In [10]:
def get_rank(rank_structure, char, position, checkpoint_interval=100000):
    """
    Get the rank (number of occurrences) of a character up to a given position using the rank structure.
    
    Args:
    rank_structure (dict): The rank data structure created by create_rank_structure
    char (str): The character to get the rank for
    position (int): The position up to which to count occurrences (exclusive)
    checkpoint_interval (int): The interval between checkpoints in the rank structure (default 100000)
    
    Returns:
    int: The number of occurrences of the character up to the given position
    """
    if char not in rank_structure:
        return 0
    
    checkpoint_index = (position - 1) // checkpoint_interval
    if checkpoint_index >= len(rank_structure[char]):
        checkpoint_index = len(rank_structure[char]) - 1
    
    last_checkpoint = rank_structure[char][checkpoint_index]
    
    remaining_start = (checkpoint_index + 1) * checkpoint_interval
    remaining_chars = bwt_last_column[remaining_start:position]
    additional_count = remaining_chars.count(char)
    
    return last_checkpoint + additional_count

## Select datastructure

The select operation is used to quickly find the position of the k-th occurrence of a character in the first column of the BWT matrix

In [11]:
def get_first_column(bwt_last_column):
    """
    Create the first column of the BWT matrix, handling newline characters.
    
    Args:
    bwt_last_column (str): The last column of the BWT
    
    Returns:
    str: The first column of the BWT matrix
    """
    # Sort the characters in the last column to get the first column
    first_column = ''.join(sorted(bwt_last_column))
    
    # Remove newline characters from the first column
    first_column = first_column.replace('\n', '')
    
    # Print detailed information about the first column
    print(f"Total length of first_column: {len(first_column)}")
    print(f"First 100 characters:")
    print(first_column[:100])
    
    # Count occurrences of each character
    char_counts = {char: first_column.count(char) for char in set(first_column)}
    print("\nCharacter counts in first column:")
    for char, count in sorted(char_counts.items()):
        print(f"{char}: {count}")
    
    return first_column

# Create the first column
first_column = get_first_column(bwt_last_column)

Total length of first_column: 151100561
First 100 characters:
$AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

Character counts in first column:
$: 1
A: 45648952
C: 29813353
G: 29865831
T: 45772424


In [12]:
def create_select_structure(first_column):
    """
    Create a select data structure for the first column of the BWT matrix, storing only intermediate positions.
    
    Args:
    first_column (str): The first column of the BWT matrix
    
    Returns:
    dict: A dictionary where keys are characters and values are lists of intermediate positions
    """
    select = {}  # Initialize an empty dictionary to store the select structure
    interval = 100000  # Define the interval for storing intermediate positions
    for i, char in enumerate(first_column):  # Iterate through each character in the first column
        if char not in select:  # If the character is not yet in the select dictionary
            select[char] = []  # Initialize an empty list for this character
        if i % interval == 0:  # Store only intermediate positions
            select[char].append(i)  # Append the current position to the list for this character
        if (i + 1) % 1000000 == 0:  # Check if we've processed a multiple of 1 million characters
            print(f"Processed {i + 1} characters. Current select structure: {select}")  # Print the current state of the select structure after every million characters
    return select  # Return the completed select structure with intermediate positions

# Create the select structure with intermediate positions
select_structure = create_select_structure(first_column)  # Use the previously defined first_column

Processed 1000000 characters. Current select structure: {'$': [0], 'A': [100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000]}
Processed 2000000 characters. Current select structure: {'$': [0], 'A': [100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000, 1100000, 1200000, 1300000, 1400000, 1500000, 1600000, 1700000, 1800000, 1900000]}
Processed 3000000 characters. Current select structure: {'$': [0], 'A': [100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000, 1100000, 1200000, 1300000, 1400000, 1500000, 1600000, 1700000, 1800000, 1900000, 2000000, 2100000, 2200000, 2300000, 2400000, 2500000, 2600000, 2700000, 2800000, 2900000]}
Processed 4000000 characters. Current select structure: {'$': [0], 'A': [100000, 200000, 300000, 400000, 500000, 600000, 700000, 800000, 900000, 1000000, 1100000, 1200000, 1300000, 1400000, 1500000, 1600000, 1700000, 1800000, 1900000, 2000000, 2100000, 2200000, 2300000, 2400000, 2500000, 260

In [13]:
# Print the first few entries of each character in the select_structure
print("First few entries for each character in select_structure:")
for char, positions in select_structure.items():  # Iterate through each character and its positions in the select_structure
    print(f"{char}: {positions[:5]}")  # Print the character and its first 5 positions (or fewer if less than 5 exist)

# Calculate and print some statistics about the select_structure
total_entries = sum(len(positions) for positions in select_structure.values())  # Calculate the total number of entries across all characters
avg_entries = total_entries / len(select_structure)  # Calculate the average number of entries per character

print(f"\nTotal number of characters in select_structure: {len(select_structure)}")  # Print the number of unique characters
print(f"Total number of entries across all characters: {total_entries}")  # Print the total number of entries
print(f"Average number of entries per character: {avg_entries:.2f}")  # Print the average number of entries per character, rounded to 2 decimal places

# Find the character with the most and least entries
max_char = max(select_structure, key=lambda x: len(select_structure[x]))  # Find the character with the most entries
min_char = min(select_structure, key=lambda x: len(select_structure[x]))  # Find the character with the least entries

print(f"\nCharacter with most entries: '{max_char}' with {len(select_structure[max_char])} entries")  # Print the character with the most entries and its count
print(f"Character with least entries: '{min_char}' with {len(select_structure[min_char])} entries")  # Print the character with the least entries and its count

# Calculate and print the memory usage of select_structure
import sys  # Import the sys module to calculate memory usage

memory_usage = sys.getsizeof(select_structure)  # Get the size of the select_structure object
for char, positions in select_structure.items():  # Iterate through each character and its positions
    memory_usage += sys.getsizeof(positions)  # Add the size of each positions list
    memory_usage += sum(sys.getsizeof(pos) for pos in positions)  # Add the size of each position integer

print(f"\nApproximate memory usage of select_structure: {memory_usage / (1024 * 1024):.2f} MB")  # Print the memory usage in MB, rounded to 2 decimal places


First few entries for each character in select_structure:
$: [0]
A: [100000, 200000, 300000, 400000, 500000]
C: [45700000, 45800000, 45900000, 46000000, 46100000]
G: [75500000, 75600000, 75700000, 75800000, 75900000]
T: [105400000, 105500000, 105600000, 105700000, 105800000]

Total number of characters in select_structure: 5
Total number of entries across all characters: 1512
Average number of entries per character: 302.40

Character with most entries: 'T' with 458 entries
Character with least entries: '$' with 1 entries

Approximate memory usage of select_structure: 0.05 MB


In [14]:
def select(select_structure, first_column, char, k):
    """
    Perform the select operation: find the position of the k-th occurrence of a character.
    
    Args:
    select_structure (dict): The select data structure created by create_select_structure
    first_column (str): The first column of the BWT matrix
    char (str): The character to select
    k (int): The occurrence number to find (1-based index)
    
    Returns:
    int: The position of the k-th occurrence of the character, or -1 if not found
    """
    if char not in select_structure or k > first_column.count(char):  # Check if the character exists and k is valid
        return -1  # Return -1 if the character or k-th occurrence doesn't exist
    
    interval = 100000  # Use the same interval as in create_select_structure
    checkpoint_index = (k - 1) // interval  # Find the nearest checkpoint before k
    
    if checkpoint_index >= len(select_structure[char]):  # If k is beyond the last checkpoint
        start_pos = select_structure[char][-1]  # Start from the last checkpoint
        count = (len(select_structure[char]) - 1) * interval  # Count up to the last checkpoint
    else:
        start_pos = select_structure[char][checkpoint_index]  # Start from the nearest checkpoint
        count = checkpoint_index * interval  # Count up to the checkpoint
    
    # Linear search from the checkpoint to the k-th occurrence
    for i in range(start_pos, len(first_column)):
        if first_column[i] == char:
            count += 1
            if count == k:
                return i
    
    return -1  # If k-th occurrence is not found

# Implement Read Alignment

In [15]:
def align_read(read, reference_sequence, first_column, bwt_last_column, bwt_map, select_structure, rank_structure):
    """
    Align a single read to the reference sequence using the BWT-based alignment algorithm.

    Args:
    read (str): The read to be aligned to the reference sequence.
    reference_sequence (str): The complete reference sequence.
    first_column (str): The first column of the BWT matrix.
    bwt_last_column (str): The last column of the BWT matrix (BWT of the reference sequence).
    bwt_map (list): The BWT mapping structure as a list for position conversion.
    select_structure (dict): The select data structure for efficient character position queries.
    rank_structure (dict): The rank data structure for efficient character count queries.

    Returns:
    int: The starting position of the alignment in the reference sequence. Returns -1 if no alignment is found.
    """
    print(f"\n--- Starting alignment for read: {read} ---")  # Log the start of alignment for a specific read
    checkpoint_interval = 100000  # Set the checkpoint interval for rank calculations to optimize performance
    print(f"Checkpoint interval set to: {checkpoint_interval}")  # Log the checkpoint interval

    # Initializing the initial band
    rightmost_char = read[-1]  # Get the rightmost character of the read
    print(f"Rightmost character of the read: '{rightmost_char}'")  # Log the rightmost character
    band_start = select(select_structure, first_column, rightmost_char, 1)  # Find the first occurrence of the rightmost character
    band_end = band_start  # Initialize band_end to band_start
    print(f"Initial band_start: {band_start}, band_end: {band_end}")  # Log initial band start and end
    for i in range(band_start + 1, len(first_column)):  # Traverse the first column to find the last occurrence
        if first_column[i] == rightmost_char:
            band_end = i
        elif first_column[i] > rightmost_char:
            break
    print(f"Initial band range for rightmost character '{rightmost_char}': [{band_start}, {band_end}]")  # Log the initial band range for the rightmost character

    for i in range(len(read) - 2, -1, -1):  # Iterate through the read from second last to first character
        current_char = read[i]  # Get the current character from the read
        print(f"\nProcessing character: '{current_char}' at position {i}")  # Log the character being processed for debugging
        
        # Find the first and last occurrence of the current character in the last column (BWT)
        first_occurrence_last_column = None  # Initialize variable to store first occurrence in last column
        last_occurrence = None  # Initialize variable to store last occurrence in last column
        print(f"Searching for '{current_char}' in the current band [{band_start}, {band_end}] of the last column")  # Log the search process
        for j in range(band_start, band_end + 1):  # Iterate through the current band in the last column
            if bwt_last_column[j] == current_char:  # Check if the character matches the current character
                if first_occurrence_last_column is None:  # If first occurrence hasn't been found yet
                    first_occurrence_last_column = j  # Set the first occurrence
                    print(f"First occurrence of '{current_char}' found at position {j} in the last column")  # Log first occurrence
                last_occurrence = j  # Update the last occurrence (will be the last matching index when loop ends)
        
        if first_occurrence_last_column is None:  # If the character was not found in the band
            print(f"Character '{current_char}' not found in the current band. Alignment failed.")  # Log alignment failure
            return -1  # Return -1 to indicate alignment failure
        
        print(f"Last occurrence of '{current_char}' found at position {last_occurrence} in the last column")  # Log last occurrence
        
        # Calculate the rank for first_occurrence_last_column and rank for last_occurrence
        first_occurrence_rank = get_rank(rank_structure, current_char, first_occurrence_last_column, checkpoint_interval)  # Get the rank of the first occurrence in the last column
        last_occurrence_rank = get_rank(rank_structure, current_char, last_occurrence + 1, checkpoint_interval) - 1  # Get the rank of the last occurrence in the last column (subtract 1 to include the last occurrence)
        print(f"Rank of first occurrence: {first_occurrence_rank}, Rank of last occurrence: {last_occurrence_rank}")  # Log the calculated ranks

        # Perform select on first column for the first_occurrence_rank and last_occurrence_rank
        new_band_start = select(select_structure, first_column, current_char, first_occurrence_rank + 1)  # Get new band start using select on first column for first_occurrence_rank
        new_band_end = select(select_structure, first_column, current_char, last_occurrence_rank + 1)  # Get new band end using select on first column for last_occurrence_rank
        
        print(f"New band range after select: [{new_band_start}, {new_band_end}]")  # Log the new band range after performing select operations

        # Check if the band has converged to a single row
        if new_band_start == new_band_end or i == 0:  # If the band has converged to a single row or we've processed all characters
            bwt_position = new_band_start  # Use the first row of the band as the BWT position
            reference_position = bwt_to_reference_position(bwt_position, bwt_map)  # Convert BWT position to reference position
            print(f"Alignment found at BWT position: {bwt_position}, Reference position: {reference_position}")  # Log the alignment position
            return reference_position  # Return the reference position where the alignment was found
        
        # Update the band for the next iteration
        band_start = new_band_start  # Update band_start for the next iteration
        band_end = new_band_end  # Update band_end for the next iteration
        print(f"Updated band range for next iteration: [{band_start}, {band_end}]")  # Log the updated band range for the next iteration
        
    # If the loop completes, return the BWT to reference position of the first row of the band
    bwt_position = band_start  # Use the first row of the band as the BWT position
    reference_position = bwt_to_reference_position(bwt_position, bwt_map)  # Convert BWT position to reference position
    print(f"Alignment found at BWT position: {bwt_position}, Reference position: {reference_position}")  # Log the alignment position
    return reference_position  # Return the reference position where the alignment was found

# Perform Read Mapping:
Iterate through all reads and align them to the reference

In [16]:
def get_reverse_complement(sequence):
    """
    Generate the reverse complement of a given DNA sequence.

    Args:
        sequence (str): The input DNA sequence.

    Returns:
        str: The reverse complement of the input sequence.
    """
    complement_dict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}  # Dictionary mapping each base to its complement
    reversed_sequence = sequence[::-1]  # Reverse the input sequence
    reverse_complement = ''.join(complement_dict.get(base, base) for base in reversed_sequence)  # Generate the complement of each base
    return reverse_complement  # Return the reverse complement sequence

In [17]:
# Initialize a list to store the aligned positions of reads
aligned_positions = []  # List to store the reference positions where reads align

# Iterate through all reads and align them to the reference
for i, read in enumerate(reads):  # Enumerate through the reads to get both index and read
    print(f"Aligning read {i + 1}/{len(reads)}")  # Log progress of alignment
    
    # Align the read using the FM-index
    aligned_position = align_read(read, reference_sequence, first_column, bwt_last_column, bwt_map, select_structure, rank_structure)  # Align the read and get its position
    
    # If alignment fails, try the reverse complement
    if aligned_position == -1:  # Check if alignment failed
        reverse_complement = get_reverse_complement(read)  # Get the reverse complement of the read
        aligned_position = align_read(reverse_complement, reference_sequence, first_column, bwt_last_column, bwt_map, select_structure, rank_structure)  # Try aligning the reverse complement
    
    # Store the aligned position (even if it's -1 for failed alignments)
    aligned_positions.append(aligned_position)  # Add the aligned position to our list
    
    # Print progress every 1000 reads
    if (i + 1) % 1000 == 0:  # Check if we've processed a multiple of 1000 reads
        print(f"Processed {i + 1} reads")  # Log progress

Aligning read 1/3066720

--- Starting alignment for read: CTTTTGTTTTTTTTTTTTTTTTTTTATAAAATATTTGGGTTTTATAAAAAAAAACACAAAAAAAAAAAAAAAAAAAAAAAACTGAACCTATCGGATGACGT ---
Checkpoint interval set to: 100000
Rightmost character of the read: 'T'
Initial band_start: 105400000, band_end: 105400000


Initial band range for rightmost character 'T': [105400000, 151100560]

Processing character: 'G' at position 99
Searching for 'G' in the current band [105400000, 151100560] of the last column
First occurrence of 'G' found at position 105400000 in the last column
Last occurrence of 'G' found at position 151100560 in the last column
Rank of first occurrence: 21997220, Rank of last occurrence: 29679800
New band range after select: [97497220, 105179800]
Updated band range for next iteration: [97497220, 105179800]

Processing character: 'C' at position 98
Searching for 'C' in the current band [97497220, 105179800] of the last column
First occurrence of 'C' found at position 97497237 in the last column
Last occurrence of 'C' found at position 105179798 in the last column
Rank of first occurrence: 19105250, Rank of last occurrence: 19444680
New band range after select: [64805250, 65144680]
Updated band range for next iteration: [64805250, 65144680]

Processing character: 'A' at position 97
S

# Exon Mapping:
- Create a data structure to represent exon locations for red and green genes
- Develop a function to check if an aligned read overlaps with exons

In [ ]:
# Define the exon locations for red and green genes
red_exons = [
    (149249757, 149249868),  # Exon location for red gene
    (149256127, 149256423),  # Exon location for red gene
    (149258412, 149258580),  # Exon location for red gene
    (149260048, 149260213),  # Exon location for red gene
    (149261768, 149262007),  # Exon location for red gene
    (149264290, 149264400)   # Exon location for red gene
]

green_exons = [
    (149288166, 149288277),  # Exon location for green gene
    (149293258, 149293554),  # Exon location for green gene
    (149295542, 149295710),  # Exon location for green gene
    (149297178, 149297343),  # Exon location for green gene
    (149298898, 149299137),  # Exon location for green gene
    (149301420, 149301530)   # Exon location for green gene
]

In [ ]:
def check_read_overlap_with_exons(read_position, read_length, exons):
    """
    Check if a read overlaps with any exon in the given list of exons.

    Args:
        read_position (int): The starting position of the read on the reference sequence.
        read_length (int): The length of the read.
        exons (list): A list of tuples, where each tuple contains the start and end positions of an exon.

    Returns:
        bool: True if the read overlaps with any exon, False otherwise.
    """
    read_end = read_position + read_length - 1  # Calculate the end position of the read
    for exon_start, exon_end in exons:  # Iterate through each exon
        if (read_position <= exon_end) and (read_end >= exon_start):  # Check for overlap
            return True  # Return True if there's an overlap
    return False  # Return False if no overlap is found

In [ ]:
# Initialize counters for red and green gene reads
red_gene_count = 0  # Counter for reads mapping to red gene exons
green_gene_count = 0  # Counter for reads mapping to green gene exons

# Process aligned reads
for i, aligned_position in enumerate(aligned_positions):
    if aligned_position != -1:  # Check if the read was successfully aligned
        read_length = len(reads[i])  # Get the length of the current read
        overlaps_red = check_read_overlap_with_exons(aligned_position, read_length, red_exons)  # Check overlap with red gene exons
        overlaps_green = check_read_overlap_with_exons(aligned_position, read_length, green_exons)  # Check overlap with green gene exons
        
        if overlaps_red and overlaps_green:  # If the read overlaps with both red and green exons
            red_gene_count += 0.5  # Add 0.5 to red gene count for ambiguous mapping
            green_gene_count += 0.5  # Add 0.5 to green gene count for ambiguous mapping
        elif overlaps_red:  # If the read overlaps only with red exons
            red_gene_count += 1  # Add 1 to red gene count for unambiguous mapping
        elif overlaps_green:  # If the read overlaps only with green exons
            green_gene_count += 1  # Add 1 to green gene count for unambiguous mapping

print(f"Total reads mapped to red gene exons: {red_gene_count}")  # Print the total count for red gene
print(f"Total reads mapped to green gene exons: {green_gene_count}")  # Print the total count for green gene

# Configuration Analysis:
- Define the possible red-green gene configurations from the presentation
- For each configuration:
  - Calculate the expected read distribution
  - Compute the probability of generating the observed read counts

In [ ]:
# Define the possible red-green gene configurations
configurations = [
    ("50% Red, 50% Green", [0.5, 0.5, 0.5, 0.5]),  # Configuration 1: 50% Red, 50% Green for exons 2,3,4,5
    ("100% Red, 0% Green", [1.0, 1.0, 0.0, 0.0]),  # Configuration 2: 100% Red for exons 2,3, 0% Red for exons 4,5
    ("33% Red, 100% Green", [0.33, 0.33, 1.0, 1.0]),  # Configuration 3: 33% Red for exons 2,3, 100% Red for exons 4,5
    ("33% Red, 33% Green", [0.33, 0.33, 0.33, 1.0])  # Configuration 4: 33% Red for exons 2,3,4, 100% Red for exon 5
]

def calculate_probability(observed_red, observed_green, expected_red_fraction):
    """
    Calculate the probability of observing the given read counts given an expected red fraction.
    
    Args:
        observed_red (float): The observed number of reads mapped to the red gene.
        observed_green (float): The observed number of reads mapped to the green gene.
        expected_red_fraction (float): The expected fraction of reads that should map to the red gene.
    
    Returns:
        float: The probability of observing the given counts under the expected distribution.
    """
    total_reads = observed_red + observed_green  # Calculate the total number of reads
    expected_red = total_reads * expected_red_fraction  # Calculate the expected number of red reads
    expected_green = total_reads * (1 - expected_red_fraction)  # Calculate the expected number of green reads
    
    # Calculate the probability using a binomial distribution
    from scipy.stats import binom
    probability = binom.pmf(k=int(observed_red), n=int(total_reads), p=expected_red_fraction)
    
    return probability  # Return the calculated probability

# Calculate probabilities for each configuration
configuration_probabilities = []  # List to store probabilities for each configuration
for config_name, red_fractions in configurations:  # Iterate through each configuration
    config_probability = 1  # Initialize the probability for this configuration
    for fraction in red_fractions:  # Calculate probability for each exon pair
        prob = calculate_probability(red_gene_count, green_gene_count, fraction)  # Calculate probability
        config_probability *= prob  # Multiply probabilities (assuming independence)
    
    configuration_probabilities.append((config_name, config_probability))  # Add to list of probabilities

# Find the most likely configuration
most_likely_config = max(configuration_probabilities, key=lambda x: x[1])  # Get configuration with highest probability

# Print results
print("Probabilities for each configuration:")
for config, prob in configuration_probabilities:
    print(f"{config}: {prob}")

print(f"\nMost likely configuration: {most_likely_config[0]} (Probability: {most_likely_config[1]})")

# Determine if the most likely configuration is associated with color-blindness
color_blindness_configs = ["100% Red, 0% Green", "33% Red, 100% Green", "33% Red, 33% Green"]  # Configurations associated with color-blindness
if most_likely_config[0] in color_blindness_configs:
    print("This configuration is associated with color-blindness.")
else:
    print("This configuration is not associated with color-blindness.")